# Etapa 2: Análise e Upload para AWS S3

**Objetivo:** Carregar os dados limpos, realizar análises para responder às perguntas de negócio e fazer o upload seguro dos resultados para um bucket na AWS S3.

In [13]:
import pandas as pd
import boto3
from dotenv import load_dotenv
from io import StringIO
import os

print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


In [14]:
# Carregamento dos Dados Limpos e Variáveis de Ambiente
# Carrega as variáveis do arquivo .env (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_BUCKET)
load_dotenv()

# Pega o nome do bucket do S3 das variáveis de ambiente
nome_bucket_s3 = os.getenv("AWS_S3_BUCKET")

# Define o caminho para o arquivo de dados limpos
caminho_dados_limpos = 'dados_processados/movimento_migratorio_limpo.csv'

# Carrega os dados
try:
    df = pd.read_csv(caminho_dados_limpos)
    print("Dados limpos carregados com sucesso!")
    if not nome_bucket_s3:
        print("AVISO: Nome do bucket S3 não encontrado no arquivo .env.")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado. Certifique-se de executar o notebook 'etl.ipynb' primeiro.")

Dados limpos carregados com sucesso!


In [15]:
# Função para Upload no S3
def upload_para_s3(df_to_upload, bucket, nome_arquivo):
    """Função para fazer o upload de um DataFrame para o S3 de forma segura."""
    print(f"Iniciando upload do arquivo '{nome_arquivo}' para o bucket '{bucket}'...")
    
    try:
        # Converte o DataFrame para um objeto CSV em memória
        csv_buffer = StringIO()
        df_to_upload.to_csv(csv_buffer, index=False)
        
        # Cria um cliente S3. Boto3 usará as credenciais carregadas por dotenv.
        s3_client = boto3.client('s3')
        
        # Faz o upload
        s3_client.put_object(Bucket=bucket, Key=nome_arquivo, Body=csv_buffer.getvalue())
        
        print(f"Upload de '{nome_arquivo}' concluído com sucesso!")
    except Exception as e:
        print(f"Falha no upload para o S3: {e}")

### Análises e Uploads

In [16]:
# Análise 1: Nacionalidade
print("\n--- Análise por Nacionalidade ---")
analise_nacionalidade = df.groupby('NACIONALIDADE')['TOTAL'].sum().reset_index().sort_values(by='TOTAL', ascending=False).head(20)
print(analise_nacionalidade)
upload_para_s3(analise_nacionalidade, nome_bucket_s3, "analise_nacionalidade.csv")

# Análise 2: UF de Atendimento
print("\n--- Análise por UF de Atendimento ---")
analise_uf = df.groupby('UF_ATENDIMENTO')['TOTAL'].sum().reset_index().sort_values(by='TOTAL', ascending=False)
print(analise_uf)
upload_para_s3(analise_uf, nome_bucket_s3, "analise_uf.csv")

# Análise 3: Classificação
print("\n--- Análise por Classificação ---")
analise_classificacao = df.groupby('CLASSIFICACAO')['TOTAL'].sum().reset_index().sort_values(by='TOTAL', ascending=False)
print(analise_classificacao)
upload_para_s3(analise_classificacao, nome_bucket_s3, "analise_classificacao.csv")


--- Análise por Nacionalidade ---
      NACIONALIDADE    TOTAL
23           BRASIL  1473473
8         ARGENTINA   207497
38            CHILE   111028
61   ESTADOS UNIDOS    83084
185         URUGUAI    52479
41         COLÔMBIA    38886
143        PORTUGAL    34507
66           FRANÇA    32965
138        PARAGUAI    27607
140            PERU    25754
39            CHINA    24643
92           ITÁLIA    24453
64        FILIPINAS    23220
189       VENEZUELA    20771
60          ESPANHA    20058
21          BOLÍVIA    18904
2          ALEMANHA    18469
146     REINO UNIDO    18180
123          MÉXICO    18006
195           ÍNDIA    10877
Iniciando upload do arquivo 'analise_nacionalidade.csv' para o bucket 'desafio-sprint-4-victor-rafael'...
Upload de 'analise_nacionalidade.csv' concluído com sucesso!

--- Análise por UF de Atendimento ---
   UF_ATENDIMENTO    TOTAL
24             SP  1424938
17             RJ   406188
21             RS   104531
16             PR    89861
6              

### Verificação

## Verificação Final

Uploads concluídos. Agora você pode verificar o bucket **[nome-do-seu-bucket]** no console da AWS para confirmar que os arquivos `analise_nacionalidade.csv`, `analise_uf.csv` e `analise_classificacao.csv` estão lá.